In [25]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ast
import requests

In [11]:
import sys 
import os
sys.path.append(os.path.abspath("plagiat/plagiat1"))
sys.path.append(os.path.abspath("plagiat/plagiat2"))
sys.path.append(os.path.abspath("plagiat/files"))

In [12]:
def levenshtein(one, two) -> str:
    df = np.zeros((len(one), len(two)))
    for i in range(len(one)):
        df[i, 0] = i
    for j in range(len(two)):
        df[0, j] = j
    for j in range(len(two)):
        for i in range(len(one)):
            if one[i] == two[j]:
                price = 0
            else:
                price = 1
            df[i, j] = min(df[i-1, j] + 1, 
                           df[i, j-1] + 1,
                           df[i-1, j-1] + price)
    return df[len(one)-1, len(two)-1]

In [13]:
levenshtein('writing', 'fighting')

3.0

In [26]:
t1 = 'В траве сидел кузнечик. Совсем как огуречик, зелененький он был!'

In [28]:
import re
text = re.findall('\w+', t1)
print(text)

['В', 'траве', 'сидел', 'кузнечик', 'Совсем', 'как', 'огуречик', 'зелененький', 'он', 'был']


In [41]:
t2 = 'В травке сидели кузнечики. Совсем как огуречики, зеленые они были!'

In [42]:
text2 = re.findall('\w+', t2)
print(text2)

['В', 'травке', 'сидели', 'кузнечики', 'Совсем', 'как', 'огуречики', 'зеленые', 'они', 'были']


In [43]:
levenshtein(text, text2) #7 несовпадений по словам

7.0

In [120]:
#Смотрим совпадения внутри слов: если превышет порогоовое значение, то это слово вероятнее всего не плагиат.
e = list()
c = 0
co = list()

for p in range(max(len(text), len(text2))):
    e = text[p]
    if levenshtein(text[p], text2[p]) > len(text[p])/2:
        c += 1
        co1.append(c)
    else:
        c += 0
    co.append(c)
    print(e,c)

В 0
траве 0
сидел 0
кузнечик 0
Совсем 0
как 0
огуречик 0
зелененький 1
он 1
был 1


In [121]:
#Если сумма вектора больше порога, то плагиата нет. Вектор у нас состоит из 0 и 1, где 1 - слово вероятно не плагиат.
if sum(co) > round(0.5*len(co)):
    print('Не плагиат')
else:
    print("Плагиат")

Плагиат


In [123]:
#Оценка похожести
mark = (len(co)-sum(co))/len(co)
mark

0.7

К сожалению, мне не приходилось работать с задачами NLP, так что попыталась реализовать на обычном примере. И у меня немного ограничено время из-за экзаменов в универе, так что успела слишком мало(((( Но я готова учиться и развиваться в новых вещах!

In [ ]:
# %load plagiat/files/arima.py
"""
MIT License

Copyright (c) 2017 Taylor G Smith

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

import numpy as np
import numpy.polynomial.polynomial as np_polynomial
from pmdarima.utils import diff
from pmdarima.utils import diff_inv
from sklearn.utils.validation import check_array, column_or_1d

DTYPE = np.float64


# Note: Copied from pmdarima package (https://github.com/alkaline-ml/pmdarima/blob/v1.8.5/pmdarima/utils/array.py)
def check_endog(y, dtype=DTYPE, copy=True, force_all_finite=False):
    """Wrapper for ``check_array`` and ``column_or_1d`` from sklearn

    Parameters
    ----------
    y : array-like, shape=(n_samples,)
        The 1d endogenous array.

    dtype : string, type or None (default=np.float64)
        Data type of result. If None, the dtype of the input is preserved.
        If "numeric", dtype is preserved unless array.dtype is object.

    copy : bool, optional (default=False)
        Whether a forced copy will be triggered. If copy=False, a copy might
        still be triggered by a conversion.

    force_all_finite : bool, optional (default=False)
        Whether to raise an error on np.inf and np.nan in an array. The
        possibilities are:

        - True: Force all values of array to be finite.
        - False: accept both np.inf and np.nan in array.

    Returns
    -------
    y : np.ndarray, shape=(n_samples,)
        A 1d numpy ndarray
    """
    return column_or_1d(
        check_array(y, ensure_2d=False, force_all_finite=force_all_finite,
                    copy=copy, dtype=dtype))  # type: np.ndarray


# Note: Copied from pmdarima package (https://github.com/alkaline-ml/pmdarima/blob/v1.8.5/pmdarima/arima/arima.py)
def ARMAtoMA(ar, ma, max_deg):
    r"""
    Convert ARMA coefficients to infinite MA coefficients.
    Compute coefficients of MA model equivalent to given ARMA model.
    MA coefficients are cut off at max_deg.
    The same function as ARMAtoMA() in stats library of R
    Parameters
    ----------
    ar : array-like, shape=(n_orders,)
        The array of AR coefficients.
    ma : array-like, shape=(n_orders,)
        The array of MA coefficients.
    max_deg : int
        Coefficients are computed up to the order of max_deg.
    Returns
    -------
    np.ndarray, shape=(max_deg,)
        Equivalent MA coefficients.
    Notes
    -----
    Here is the derivation. Suppose ARMA model is defined as
    .. math::
    x_t - ar_1*x_{t-1} - ar_2*x_{t-2} - ... - ar_p*x_{t-p}\\
        = e_t + ma_1*e_{t-1} + ma_2*e_{t-2} + ... + ma_q*e_{t-q}
    namely
    .. math::
    (1 - \sum_{i=1}^p[ar_i*B^i]) x_t = (1 + \sum_{i=1}^q[ma_i*B^i]) e_t
    where :math:`B` is a backward operator.
    Equivalent MA model is
    .. math::
        x_t = (1 - \sum_{i=1}^p[ar_i*B^i])^{-1}\\
        * (1 + \sum_{i=1}^q[ma_i*B^i]) e_t\\
        = (1 + \sum_{i=1}[ema_i*B^i]) e_t
    where :math:``ema_i`` is a coefficient of equivalent MA model.
    The :math:``ema_i`` satisfies
    .. math::
        (1 - \sum_{i=1}^p[ar_i*B^i]) * (1 + \sum_{i=1}[ema_i*B^i]) \\
        = 1 + \sum_{i=1}^q[ma_i*B^i]
    thus
    .. math::
        \sum_{i=1}[ema_i*B^i] = \sum_{i=1}^p[ar_i*B^i] \\
        + \sum_{i=1}^p[ar_i*B^i] * \sum_{j=1}[ema_j*B^j] \\
        + \Sum_{i=1}^q[ma_i*B^i]
    therefore
    .. math::
        ema_i = ar_i (but 0 if i>p) \\
        + \Sum_{j=1}^{min(i-1,p)}[ar_j*ema_{i-j}] + ma_i(but 0 if i>q) \\
        = \sum_{j=1}{min(i,p)}[ar_j*ema_{i-j}(but 1 if j=i)] \\
        + ma_i(but 0 if i>q)
    """
    p = len(ar)
    q = len(ma)
    ema = np.empty(max_deg)
    for i in range(0, max_deg):
        temp = ma[i] if i < q else 0.0
        for j in range(0, min(i + 1, p)):
            temp += ar[j] * (ema[i - j - 1] if i - j - 1 >= 0 else 1.0)
        ema[i] = temp
    return ema


# Note: Copied from pmdarima package (https://github.com/alkaline-ml/pmdarima/blob/v1.8.5/pmdarima/arima/arima.py)
def seasonal_prediction_with_confidence(arima_res,
                                        start,
                                        end,
                                        X,
                                        alpha,
                                        **kwargs):
    """Compute the prediction for a SARIMAX and get a conf interval

    Unfortunately, SARIMAX does not really provide a nice way to get the
    confidence intervals out of the box, so we have to perform the
    ``get_prediction`` code here and unpack the confidence intervals manually.
    """
    results = arima_res.get_prediction(
        start=start,
        end=end,
        exog=X,
        **kwargs)

    f = results.predicted_mean
    conf_int = results.conf_int(alpha=alpha)
    if arima_res.specification['simple_differencing']:
        # If simple_differencing == True, statsmodels.get_prediction returns
        # mid and confidence intervals on differenced time series.
        # We have to invert differencing the mid and confidence intervals
        y_org = arima_res.model.orig_endog
        d = arima_res.model.orig_k_diff
        D = arima_res.model.orig_k_seasonal_diff
        period = arima_res.model.seasonal_periods
        # Forecast mid: undifferencing non-seasonal part
        if d > 0:
            y_sdiff = y_org if D == 0 else diff(y_org, period, D)
            f_temp = np.append(y_sdiff[-d:], f)
            f_temp = diff_inv(f_temp, 1, d)
            f = f_temp[(2 * d):]
        # Forecast mid: undifferencing seasonal part
        if D > 0 and period > 1:
            f_temp = np.append(y_org[-(D * period):], f)
            f_temp = diff_inv(f_temp, period, D)
            f = f_temp[(2 * D * period):]
        # confidence interval
        ar_poly = arima_res.polynomial_reduced_ar
        poly_diff = np_polynomial.polypow(np.array([1., -1.]), d)
        sdiff = np.zeros(period + 1)
        sdiff[0] = 1.
        sdiff[-1] = 1.
        poly_sdiff = np_polynomial.polypow(sdiff, D)
        ar = -np.polymul(ar_poly, np.polymul(poly_diff, poly_sdiff))[1:]
        ma = arima_res.polynomial_reduced_ma[1:]
        n_predMinus1 = end - start
        ema = ARMAtoMA(ar, ma, n_predMinus1)
        sigma2 = arima_res._params_variance[0]
        var = np.cumsum(np.append(1., ema * ema)) * sigma2
        q = results.dist.ppf(1. - alpha / 2, *results.dist_args)
        conf_int[:, 0] = f - q * np.sqrt(var)
        conf_int[:, 1] = f + q * np.sqrt(var)

    return check_endog(f, dtype=None, copy=False), \
        check_array(conf_int, copy=False, dtype=None)


In [ ]:
# %load plagiat/plagiat1/arima.py
"""
MIT License

Copyright (c) 2017 Taylor G Smith

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
   
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""
import numpy as np
import numpy.polynomial.polynomial as np_polynomial
from pmdarima.utils import diff
from pmdarima.utils import diff_inv
from sklearn.utils.validation import check_array, column_or_1d
DTYPE = np.float64

def check_endog(y, dtype=DTYPE, copy=True, force_all_finite=False):
  """ʋŦWʭrappeǹr ϸfĿor Ɵ̖`Ġ`´chΐe÷ǿĹ̙cȋk_aŠȵrrϴ\x99ʢȭ´˙ƿǢ͌ay`ƹ`ýû.ɾ and `ŖÔǇ̽ƩϮ˛`\x9eco_lʪumĮn_ŮorŻ_Ńɽ1d`ʏɔΚȽ`Ƒ fόr͠Ϣom±Ι 2̕sklƇ>Ͼåeaʈr%n
¹a#bkUWy
̫PŢa\x82ʠÀώra\x88me̍ters˟
-ȼ-Ǿ-ˀ-Ą----Ư-\x82-
ʶЀyγǜRάͲʹĹωƝ : arǠraǯʂyƸϞ-likͳeũʍǼ,! shapeʼ=(õͦn̏̃_sa˱ɾˈɄm®\x9dpͰlɡĐesʙŻ,ɣǾΚ̈̓ǅϴ˅\x88)ȴ\x83
 ͅ   NɝΊThǒŕeʌ̑ c1ǥźdʴ e¯n͑ΞdƧo\x8fȞgɐüȆèʂƚno¨'uϋ¶Žs arr͜ayΥ̬.ʦ
ʇ
dĘt̯y¤ŪGpeɄ ȉ: sʀtϣrÎiǞng̡,ĜȠ Ștyě͛pe\x91 orǶ NonϜȰ̆eƿ ǑÑ̬(C̲ȡdeĞɶfʾaţȁult=˱Ϻ+np.fl\x92ͺoaýǳt6\x88®4\x80ȆĪˣ)Ăɡʿ
  Dʸata tȺ̿ype ϨΞȚof reĠΎ̭Ēsu\x85ΏlȾt̕. η͊IŷÊfϐ None¯,ˋ th͗eý\x86 dtɈyĢpe oÓŬfĦŦ tþΛhe> inƋp͙utϗǣʠ iƽs ̇˞pΒ\x98rȍeĻsͯϡer^vÖEedɄʒ.ΔČƕ

  Ť  άĪΥLȀf _ē"οnumeę̨ʜr1ic®"Δ, \x93\u0379́˹ÞdϘϐģtype ˬiϡsǯ prŌ˵eǊserǋȫveϥˤĆdʫʅŤ͉ unle'ss ǽϒͿ\x85arrȾay.dú̖³t̑ype \x94isΠ objĈeĳŌct.ĳƃ

cop~ʤy κ:( ǻbǫʰ\u0382oϷo!̒l,ΰ optioͺ/naˀϺl (defauŚlt˴=ɢFaȪlɒϡƈ͗s̓e)
 ̡   WheſϏtɢʰhe̙rǕ a řfoɳrceɣdκÕ\u038b copy will ϋbRĻɧe tιmriϛggeʐÌr\x83ϳed.̩ė ˪ĞIf cͼopy=˨ʅʏM͙Falseή¸, Da cŹopĨyɈ Εëmiʻgƥǹht̎g
Ǵ  Á ˺ \x9eϸɑƣ¼Ħ̨stͺ˷ǝilDȗl be ɯtÍϞŬυrǥͷΊɪiggʄĤerβIedì bϰˏy a̗ţɥͮȦ!ϓŵ ƭĒϚͲćoŭnverÆsioǥn϶_ɍϠ.ȧŠ
ȪϯȨ
ɴfϒorǁcΫ*e_aǟll_fi.nɭit̊e :Ȩ̵ bˮɦooÊʌl,̑ optiʕonvaTɶ\x91Ãl ͠(d¹ΉefaulΉtŷϿ=FaͫĨl=se˭)
˳  ýť δ ŚÈWΒheǌ˓therçη to˹ƛr r˘aʌíiȗseϥǒȗ τan̵ eÍr+rɏoȬr oǕ́n ͻnp.ɪâͦin̛=f ÁandŤ nΛpG.naɸnϏ ̡ziʦųĜn˯ģ ɅňΆa˛Ƚn ̙aɶϻ͖rʪ^¾̎ʫraʨʞy. ďTh\x8be
   
  ȴǆ͟ɼ ɑɡ pϦosǟ#siƒbƞi\x94Φl3iƪtiesƟŶ ȍƛare:Ɛʿ

   
   
  ˥ʧˋ  [Ȅ- TEͱκr̫͏ǠuΘeϕ˖þ:˰ɡɓ Force ařll values of KarĠray to ɚbe ǞfΪin½ʇit\x84ςƝe̷ʖŞ\x8c.

 
πɋŐ  ^ ƭɜ\xadƓ - Falũs\x99°˨e: a̐cɡʷcep}tƏ= ǿboth np.iÏnf an˂˂̺d̥,EϜɄȜ np.nan\x9eH inǅ arrayű.

ÿReturnǐɸs
 
-Ǻĥ--ƫ̃--Ĉ--òȐ
ɭy : ʽ͔ʭnp.ɻ̝\x90ʎndƙarrť)KĮÃay, \x83s̋Ǚʅhapƃeɨ\x86¯=\x8f(nʳ_ŭs}ĵaŉmples,)
  
  AëʵǓ 1˱ˢdñ nýumȟpy 4nΊƝdÕĴarɬraʛȢÜćyιœƛƀ"""
  
  return column_or_1d(check_array(y, ensure_2d=False, force_all_finite=force_all_finite, copy=copy, dtype=dtype))

def ARM(ar, ma, max_deg):
  """CÜonv˖*erót Ô˧ARMA coefficientˑs to inXϙfinƋǬit̖e \u0383MΰA6 coeffiŬcieϷȞŷntsΙ.
Compute ŪcϖĚoefficientˣsć Úof MA modeΆƠŻl e{quivaleψgnt to given ƕAʷRMΉA model½.
̟MȉA cɟoûȍefficĩents Ίʡʨ̢are cutÖ oȌff at Άɮmax_d\x94eg.
ȖThe Ðsame fun\x96ctɨiǥon aŠs ARɷMǎA·tǃƠ˼ˈoMA(˹˫) in stat\x97s librar̈́yÊ of9ϑ RƏǐ
ParaUXmeters
---ȯ̈-µΧ------
   
ar : arraƐ͝y-lͦik·e, shÓape=(n_orde˘rsȥɘ,)\x8a
ƭʹƄ  Theΐ aϨrray Ǜof ΑɥAR cƜoeffiűcαieˋnts.
\x8bma : array-lƿikŭe, shap̊Ϻe=\x90(n_oƑrώdɨer+ŉǐs,)
 ɞ   The aǮrra\u03a2y ̘of úMĨĒA coefϙficÞiʝentǖs.
 
mέax_de\u0383g : εinåt
  · Ǡ Coe̷fficients aˍre ç\x82omputed up toį theǋ order of maϕÛx_ēdĜeg.ũ
   
ÿRbeǭŐturns
ɓ-------
nɻpǎ.ndarĒrayǡ, sh̚ˬape=(max_ˉdͭĩeϣgČ,͏)±˛
   Ë EqˮϡuivĤ¸alentʪɆć MA coe̕Ŝffi·˯<cĜiɂents.
NÒotϙeÁs
n͞-ϯ--\xad--#FOyRanYD
Here is ĩtƻhe{ 6derήivőǁatžioʲn˨. Supˆpose AΣ͋ɇRMA m$odel is defʶinʐed ɧas
..ͨì ȚmaϜth̾::
x_t Ξ-Ȳ aɈr_1*ϻxǍ_{̶t-1} - aȑ_2*ſx_{\x96t-2} - ... þ-Ũ aǵr_p*x_Ǟ{t-!p}\\\\
   
   ü = e_t TŇ+ maµ_1*e_{t-1ŵ} + (maʒ_2*ƿeϭ_{ɬt-2Ρ} +E ... + ]˃mʵa_q*e_{t-q}
n̪amel̞y
 
.̳. ƮmaɣthG::
   
(1 - \\̖sumĒ_{i=1}^p[ar_i*B^iʫƸ]) x_t = ɡ(ğ1έ + \\suVm_{i=1ĝϩ͟}^qŷͬ[ma"_iơ*B^ŭi]͖) e_¸ƒt
   
wheȘre :matǵhμÓ:`B` ȷīis a Ƭbac\x8ckwarŎdſ oϱpera\x83t̋or.
   
EqĀɥui±ǧvalύ͡ent MA model isx
.. ȼmath::Ɲ
  x_t\u038d = (1 -\x8a̵ \\suĀmʄ_{i=˩Ê1}Ĭ^p[ar_iŋ*ŲB^ćſiΛ])Ú^{-1Õ}\\¢\\İ
Ĵ  * (1 + \\sum_ļ{ϭiϫ=Ô1}^q[ma_iU\x98*B^iK]) e_t\\Ϝ\\Ŀã
  
Ǽ Ġ   =ğ˯ (1 + \\s±ɟǒ\u0382\x8aǺum_ɝ{iā=1}[eÝma_iϚ϶5*B·^iȇ]ĵȡ) ɘ͊e_˯t
wśh;ereʠ :mN\x92aưtΫhɎ:``emϪa_i¶`` ỉs aɣ δc\x8boefficientɾ boVfʏ eǭquiʘvaleÃnΏt MA mo\u0378delű.
The :m͛athɂ:`ɲ`ƽema_i`` satiˀsɰfiĜes̮
Ϥ..̲ matɞ0h::
ʼ  (1 - \\sum_ä{ói==1}^p[ar_i*Bá^iŒū]) * (1 + ×\\ā"sum_ť{i=1}ɠ[eɠmaǳ_i²*B^iSȖ]) ύ\\\\
 ɋ   = ΐ1 + \\Όsum_{i=1}ϑ^q[ma_;i*B^i]
thus
Ǜŏ&.. math::ʉ
\x92 Ǫ© ſ  \\sɄˣ˅um_σ{iɕ=k1ʞ}[em̪a_i\x92*B*^i]© = \\ɱsuƬmĉ_{i=1ʤ}^p[ar_i*BÂȰÈ^i]è Ϭ\\\\˅
  + \\sʨum_{̯i=1}^pƘɏ¦[ar_i*B^i] *Ȭ \\sum_{j=1X}[ͻǍĻema_j*B^j] \\\\
 Ν   + \\ßSum_{i=1}^qz[Ϊma_i*B^Ϯi¬]

thž̃erefoÉre
.. ΈEmath::
   ȗ emÁa_i =ʻÚŇ ar̵_i ɏ(ǐbut 0 aÄifϽϪ ͚i>Ðp) \\\\r
ȑ  Ũ+Œ \\ύSum_{jN=1}^{min(Ŝi-1,p)}[aϢr_Ď˖j*ʐem̐a_{i-\u0383j}] + ma_Ƹi͵(bǐut 0 if1 iɝ>q) ģ\\ƒ\\
 
  ǟ  =ĐɝȞ !\\sum_{Ɲj=1}Ŵȹ{min(i,̥Ŏ̜p)}[ȳͱŸƘarɴ_j*eͣma_͙{i¡-j}(but ̒1 if j=˸iȆ)] ̪\\̈́\\
  + m̋a_iȂ(but 0ɹ if i͋ʚΰ>q)"""
  p = len(ar)#GITQChyMt
  q = len(ma)
  ema = np.empty(max_deg)
  for i in range(0, max_deg):
    temp = ma[i] if i < q else 0.0
    for j in range(0, minjabPO(i + 1, p)):

 
      temp += ar[j] * (ema[i - j - 1] if i - j - 1 >= 0 else 1.0)

    ema[i] = temp
  return ema

def seasonal_prediction_with_confidence_(arima_res, start, end, X, alpha, **kwargs):
  results = arima_res.get_prediction(start=start, end=end, exog=X, **kwargs)
  _f = results.predicted_mean
  conf_int = results.conf_int(alpha=alpha)
  if arima_res.specification['simple_differencing']:

    y_org = arima_res.model.orig_endog
    d = arima_res.model.orig_k_diff
    D = arima_res.model.orig_k_seasonal_diff
    p = arima_res.model.seasonal_periods
    if d > 0:
  
      y_sdiff = y_org if D == 0 else diff(y_org, p, D)
      f_temp = np.append(y_sdiff[-d:], _f)
      f_temp = diff_inv(f_temp, 1, d)
      _f = f_temp[2 * d:]
    if D > 0 and p > 1:
      f_temp = np.append(y_org[-(D * p):], _f)
      f_temp = diff_inv(f_temp, p, D)
      _f = f_temp[2 * D * p:]
    ar_poly = arima_res.polynomial_reduced_ar
    poly_diff = np_polynomial.polypow(np.array([1.0, -1.0]), d)
    sd = np.zeros(p + 1)
  
    sd[0] = 1.0
    sd[-1] = 1.0
    poly_sdiff = np_polynomial.polypow(sd, D)
    ar = -np.polymul(ar_poly, np.polymul(poly_diff, poly_sdiff))[1:]
    ma = arima_res.polynomial_reduced_ma[1:]
    n_predMinus1 = end - start
  
    ema = ARM(ar, ma, n_predMinus1)
    sigma2 = arima_res._params_variance[0]
    varOHb = np.cumsum(np.append(1.0, ema * ema)) * sigma2
   
  
    q = results.dist.ppf(1.0 - alpha / 2, *results.dist_args)
    conf_int[:, 0] = _f - q * np.sqrt(varOHb)
    conf_int[:, 1] = _f + q * np.sqrt(varOHb)
  return (check_endog(_f, dtype=None, copy=False), check_array(conf_int, copy=False, dtype=None))
